In [1]:
import pandas as pd
import os
from sklearn.metrics import classification_report
from tqdm import tqdm
import itertools
import warnings
import sys
sys.path.append('../src')
import itertools

from models.classification_methods import get_classification_report
warnings.filterwarnings("ignore")

In [2]:
def generate_latex_with_multirow_and_bold(df):
    latex_code = ''
    latex_code += '\\begin{table}[H]'
    latex_code += '\\scriptsize'
    latex_code += '\\centering'
    latex_code += "\\begin{tabular}{ll|rrrrrrr}\n\\toprule\n"
    latex_code += "input & classifier & Church & Bolsonaro & Hydrox. & Sinovac & Globo TV & Lula & overall \\\\ \n\\midrule\n"

    last_input = None
    multirow_count = 0

    for input_value in df['input'].unique():
        subset = df[df['input'] == input_value]
        max_overall_idx = subset['overall'].idxmax()

        for i, row in subset.iterrows():
            if row['input'] == last_input:
                latex_code += "& "
                multirow_count += 1
            else:
                if multirow_count > 0:
                    latex_code = latex_code.replace(f"multirow{{{multirow_count}}}", f"multirow{{{multirow_count + 1}}}", 1)
                if last_input is not None:
                    latex_code += "\\cmidrule(lr){1-9}\n"
                latex_code += f"\\multirow{{{1}}}{{*}}{{{row['input']}}} & "
                multirow_count = 1

            if i == max_overall_idx:
                row_data = [f"\\textbf{{{row[col]:.2f}}}" if col not in ['input', 'classifier'] else f"\\textbf{{{row[col]}}}" for col in df.columns[1:]]
                latex_code += " & ".join(row_data) + " \\\\ \n"
            else:
                latex_code += " & ".join([f"{row[col]:.2f}" if isinstance(row[col], float) else str(row[col]) for col in df.columns[1:]]) + " \\\\ \n"
            
            last_input = row['input']

    if multirow_count > 0:
        latex_code = latex_code.replace(f"multirow{{{multirow_count}}}", f"multirow{{{multirow_count + 1}}}", 1)

    latex_code += "\\cmidrule(lr){1-9}\n"
    latex_code += "\\bottomrule\n\\end{tabular}"
    latex_code += "\caption{F1 macro results}"
    latex_code += '\label{table:results_f1_macro}\n'
    latex_code += '\end{table}'

    return latex_code

In [3]:
test_results_path = '../reports/test_results/'

list_df_t = os.listdir(test_results_path)
list_df_t.sort()
list_df_t

['DummyClassifier_bo_concat_Texts_Timeline_concat_Texts_Timeline_test_results.csv',
 'DummyClassifier_bo_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_bo_users_Stance_test_results.csv',
 'DummyClassifier_bo_users_Timeline_test_results.csv',
 'DummyClassifier_cl_concat_Texts_Timeline_concat_Texts_Timeline_test_results.csv',
 'DummyClassifier_cl_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_cl_users_Stance_test_results.csv',
 'DummyClassifier_cl_users_Timeline_test_results.csv',
 'DummyClassifier_co_concat_Texts_Timeline_concat_Texts_Timeline_test_results.csv',
 'DummyClassifier_co_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_co_users_Stance_test_results.csv',
 'DummyClassifier_co_users_Timeline_test_results.csv',
 'DummyClassifier_gl_concat_Texts_Timeline_concat_Texts_Timeline_test_results.csv',
 'DummyClassifier_gl_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_gl_users_Stance_test_results.csv',
 'Dummy

In [4]:
# Target list
target_list = [
    'ig',
    'bo', 
    'cl', 
    'co', 
    'gl', 
    'lu'
]

dict_cp = {
    'cl':'Hydrox.',
    'lu':'Lula',
    'co':'Sinovac',
    'ig':'Church',
    'gl':'Globo TV',
    'bo':'Bolsonaro',
}

names = list(dict_cp.values())
names

['Hydrox.', 'Lula', 'Sinovac', 'Church', 'Globo TV', 'Bolsonaro']

## Create complete table

Get all the combinations of Stacking:

In [5]:
my_list = [
    'Texts', 'Timeline', 'Texts',
    'BoM', 'BoF', 'BoFr'
]

# Remove duplicados mantendo a ordem
unique_my_list = []
seen = set()
for item in my_list:
    if item not in seen:
        unique_my_list.append(item)
        seen.add(item)

all_combinations = []
for r in range(2, len(unique_my_list) + 1):
    all_combinations.extend(itertools.combinations(unique_my_list, r))



tuples_stacking_exps = []
clf_name = 'LogisticRegression'
preprocess_name = '-'
for comb in all_combinations:
    
    str_cols = "_".join(comb)
    
    exp_name = 'Stacking ' + " + ".join(comb)
    
    
    filename = 'Ensemble_LogisticRegression_{target}_' + str_cols + '_test_results.csv'
    tuples_stacking_exps.append((exp_name, preprocess_name, clf_name, filename))

In [6]:

PATH_BEST_BoM = ""
PATH_BEST_BoF = ""

# (vectorizer,estimator, path_sring) 
results_tuples_stance = [
    # Stance
    ("Stance", "-" ,"dummy", "DummyClassifier_{target}_users_Stance_test_results.csv"),
    ("Stance", "tf-idf" ,"xgb", "XGBClassifier_TfidfVectorizer_{target}_users_Stance_test_results.csv"),
    ("Stance", "bertabaporu-base" ,"xgb", "bertimbau_xgb_{target}_users_emb_Stance_test_results.csv"),
    ("Stance", "-" ,"bertabaporu-base", "bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_Stance_test_results.csv"),
    ("Stance", "-",  "llama3:7b zero-shot", "llama3_{target}_Stance_prompt2_Stance_test_results.csv"),
    
    # Texts
    ("Texts", "-" ,"dummy", "DummyClassifier_{target}_top_mentioned_timelines_Texts_test_results.csv"),
    ("Texts", "tf-idf" ,"xgb", "XGBClassifier_TfidfVectorizer_{target}_top_mentioned_timelines_Texts_test_results.csv"),
    ("Texts", "bertabaporu-base" ,"xgb", "bertimbau_xgb_{target}_top_mentioned_timelines_emb_Texts_test_results.csv"),
    ("Texts", "-" ,"bertabaporu-base", "bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_Texts_test_results.csv"),
    ("Texts", "-" ,"bertabaporu-base (R)", "bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_scored_Texts_test_results.csv"),
    ("Texts", "-", "llama3:7b zero-shot [5] (R)", "llama3_{target}_filtered_Texts5_prompt2_Texts_test_results.csv"),
    ("Texts", "-", "llama3:7b zero-shot [10] (R)", "llama3_{target}_filtered_Texts10_prompt2_Texts_test_results.csv"),
    ("Texts", "-", "llama3:7b zero-shot [15] (R)", "llama3_{target}_filtered_Texts15_prompt2_Texts_test_results.csv"),
    
    # Timeline
    ("Timeline", "-" ,"dummy", "DummyClassifier_{target}_users_Timeline_test_results.csv"),
    ("Timeline", "tf-idf" ,"xgb", "XGBClassifier_TfidfVectorizer_{target}_users_Timeline_test_results.csv"),
    ("Timeline", "bertabaporu-base" ,"xgb", "bertimbau_xgb_{target}_users_emb_Timeline_test_results.csv"),
    ("Timeline", "-" ,"bertabaporu-base", "bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_Timeline_test_results.csv"),
    ("Timeline", "-" ,"bertabaporu-base (R)", "bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_scored_Timeline_test_results.csv"),
    ("Timeline", "-", "llama3:7b zero-shot [5] (R)", "llama3_{target}_filteredTimeline5_prompt2_Timeline_test_results.csv"),
    ("Timeline", "-", "llama3:7b zero-shot [10] (R)", "llama3_{target}_filteredTimeline10_prompt2_Timeline_test_results.csv"),
    ("Timeline", "-", "llama3:7b zero-shot [15] (R)", "llama3_{target}_filteredTimeline15_prompt2_Timeline_test_results.csv"),
    
    # Bag of xxx
    ('Bag of Friends', 'tf-idf', 'LogisticRegression', "TfidfVectorizer_SelectKBest_LogisticRegression_{target}_BagOfFriends_test_results.csv"),
    ('Bag of Followers', 'tf-idf', 'LogisticRegression', "TfidfVectorizer_SelectKBest_LogisticRegression_{target}_BagOfFollowers_test_results.csv"),
    ('Bag of Mentions', 'tf-idf', 'LogisticRegression', "TfidfVectorizer_SelectKBest_LogisticRegression_{target}_BagOfMentions_test_results.csv"),
    
    # Ensembles
    # ("Ensemble Texts + Timeline", "-", "LogisticRegression", "Ensemble_LogisticRegression_{target}_Texts_Timeline_test_results.csv"),
    # ("Ensemble Texts + Stance", "-", "LogisticRegression", "Ensemble_LogisticRegression_{target}_Texts_Stance_test_results.csv"),
    # ("Ensemble Stance + Timeline", "-", "LogisticRegression", "Ensemble_LogisticRegression_{target}_Stance_Timeline_test_results.csv"),
    # ("Ensemble Stance + Timeline + Texts", "-", "LogisticRegression", "Ensemble_LogisticRegression_{target}_Stance_Timeline_Texts_test_results.csv"),
    ("concat_Texts_Timeline", "tf-idf", "xgb", "XGBClassifier_TfidfVectorizer_{target}_concat_Texts_Timeline_concat_Texts_Timeline_test_results.csv")
]

In [7]:
results_tuples_stance = results_tuples_stance + tuples_stacking_exps

In [8]:
results_tuples_stance

[('Stance',
  '-',
  'dummy',
  'DummyClassifier_{target}_users_Stance_test_results.csv'),
 ('Stance',
  'tf-idf',
  'xgb',
  'XGBClassifier_TfidfVectorizer_{target}_users_Stance_test_results.csv'),
 ('Stance',
  'bertabaporu-base',
  'xgb',
  'bertimbau_xgb_{target}_users_emb_Stance_test_results.csv'),
 ('Stance',
  '-',
  'bertabaporu-base',
  'bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_Stance_test_results.csv'),
 ('Stance',
  '-',
  'llama3:7b zero-shot',
  'llama3_{target}_Stance_prompt2_Stance_test_results.csv'),
 ('Texts',
  '-',
  'dummy',
  'DummyClassifier_{target}_top_mentioned_timelines_Texts_test_results.csv'),
 ('Texts',
  'tf-idf',
  'xgb',
  'XGBClassifier_TfidfVectorizer_{target}_top_mentioned_timelines_Texts_test_results.csv'),
 ('Texts',
  'bertabaporu-base',
  'xgb',
  'bertimbau_xgb_{target}_top_mentioned_timelines_emb_Texts_test_results.csv'),
 ('Texts',
  '-',
  'bertabaporu-base',
  'bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_T

In [9]:


list_results = []
for text_col, vectorizer, estimator, path_results in results_tuples_stance:
    
    list_cr = []
    
    for target in target_list:
        
        
        path = test_results_path + path_results.format(target = target)
        df_results = pd.read_csv(path)
        df_results_or = df_results.copy()
        
        # get classification report df
        df_classification_report = get_classification_report(df_results.test, df_results.pred, cr_args = {})
        
        # create multindex
        column_indexes = [(metric,dict_cp[target]) for metric in df_classification_report.columns]
        multi_index_cols = pd.MultiIndex.from_tuples(column_indexes, names=['metric', 'target'])
        rows_indexes = [(text_col, vectorizer, estimator, cl) for cl in df_classification_report.index]
        multi_index_rows = pd.MultiIndex.from_tuples(rows_indexes, names=['text_col','vectorizer', 'estimator', 'class'])
        df_classification_report.columns = multi_index_cols
        df_classification_report.index = multi_index_rows
        
        # print(text_col, vectorizer, estimator,target)
        # print(path)
        # display(df_classification_report)
        
        list_cr.append(df_classification_report)
        
    df_results = pd.concat(list_cr, axis = 1)
    
    list_results.append(df_results)
    
df_results_final = pd.concat(list_results)

In [10]:
df_results_final

metric                                                                                  precision  \
target                                                                                     Church   
text_col                                     vectorizer estimator          class                    
Stance                                       -          dummy              against       0.565943   
                                                                           accuracy      0.565943   
                                                                           weighted avg  0.320292   
                                                                           macro avg     0.282972   
                                                                           for           0.000000   
...                                                                                           ...   
Stacking Texts + Timeline + BoM + BoF + BoFr -          LogisticRegression against       0.729050   
                                                                           accuracy      0.707846   
                                                                           weighted avg  0.706175   
                                                                           macro avg     0.702699   
                                                                           for           0.676349   

metric                                                                                     recall  \
target                                                                                     Church   
text_col                                     vectorizer estimator          class                    
Stance                                       -          dummy              against       1.000000   
                                                                           accuracy      0.565943   
                                                                           weighted avg  0.565943   
                                                                           macro avg     0.500000   
                                                                           for           0.000000   
...                                                                                           ...   
Stacking Texts + Timeline + BoM + BoF + BoFr -          LogisticRegression against       0.769912   
                                                                           accuracy      0.707846   
                                                                           weighted avg  0.707846   
                                                                           macro avg     0.698417   
                                                                           for           0.626923   

metric                                                                                   f1-score  \
target                                                                                     Church   
text_col                                     vectorizer estimator          class                    
Stance                                       -          dummy              against       0.722814   
                                                                           accuracy      0.565943   
                                                                           weighted avg  0.409072   
                                                                           macro avg     0.361407   
                                                                           for           0.000000   
...                                                                                           ...   
Stacking Texts + Timeline + BoM + BoF + BoFr -          LogisticRegression against       0.748924   
                                                                           accuracy      0.707846   
                                                           

In [11]:
df_results_final.to_excel("../reports/table_complete_results.xlsx")

## Create table f1

In [12]:
mask_f1 = [True if  "f1-score" in col else False for col in df_results_final.columns]
mask_macro = [True if  "for" in col else False for col in df_results_final.index]

f1_df = df_results_final.loc[mask_macro,mask_f1]
f1_df[('f1-score','overall')] = f1_df.mean(axis=1)

f1_df

metric                                                                                            f1-score  \
target                                                                                              Church   
text_col                                     vectorizer       estimator                    class             
Stance                                       -                dummy                        for    0.000000   
                                             tf-idf           xgb                          for    0.654762   
                                             bertabaporu-base xgb                          for    0.824268   
                                             -                bertabaporu-base             for    0.844622   
                                                              llama3:7b zero-shot          for    0.734426   
Texts                                        -                dummy                        for    0.000000   
                                             tf-idf           xgb                          for    0.506438   
                                             bertabaporu-base xgb                          for    0.513570   
                                             -                bertabaporu-base             for    0.458937   
                                                              bertabaporu-base (R)         for    0.458234   
                                                              llama3:7b zero-shot [5] (R)  for    0.491525   
                                                              llama3:7b zero-shot [10] (R) for    0.508539   
                                                              llama3:7b zero-shot [15] (R) for    0.432749   
Timeline                                     -                dummy                        for    0.000000   
                                             tf-idf           xgb                          for    0.647887   
                                             bertabaporu-base xgb                          for    0.596273   
                                             -                bertabaporu-base             for    0.656766   
                                                              bertabaporu-base (R)         for    0.666667   
                                                              llama3:7b zero-shot [5] (R)  for    0.559546   
                                                              llama3:7b zero-shot [10] (R) for    0.479339   
                                                              llama3:7b zero-shot [15] (R) for    0.509636   
Bag of Friends                               tf-idf           LogisticRegression           for    0.626712   
Bag of Followers                             tf-idf           LogisticRegression           for    0.538776   
Bag of Mentions                              tf-idf           LogisticRegression           for    0.582329   
concat_Texts_Timeline                        tf-idf           xgb                          for    0.647541   
Stacking Texts + Timeline                    -                LogisticRegression           for    0.650794   
Stacking Texts + BoM                         -                LogisticRegression           for    0.567850   
Stacking Texts + BoF                         -                LogisticRegression           for    0.602362   
Stacking Texts + BoFr                        -                LogisticRegression           for    0.504386   
Stacking Timeline + BoM                      -                LogisticRegression           for    0.652174   
Stacking Timeline + BoF                      -                LogisticRegression           for    0.665480   
Stacking Timeline + BoFr                     -                LogisticRegression           for    0.647887   
Stacking BoM + BoF                           -                LogisticRegression           for    0.626050   
Stacking BoM + BoFr                     

In [13]:
f1_df.sort_values([('f1-score','overall')])

metric                                                                                            f1-score  \
target                                                                                              Church   
text_col                                     vectorizer       estimator                    class             
Stance                                       -                dummy                        for    0.000000   
Texts                                        -                dummy                        for    0.000000   
Timeline                                     -                dummy                        for    0.000000   
Texts                                        -                bertabaporu-base             for    0.458937   
                                                              bertabaporu-base (R)         for    0.458234   
                                                              llama3:7b zero-shot [5] (R)  for    0.491525   
                                             bertabaporu-base xgb                          for    0.513570   
                                             tf-idf           xgb                          for    0.506438   
Timeline                                     -                llama3:7b zero-shot [10] (R) for    0.479339   
Stacking Texts + BoFr                        -                LogisticRegression           for    0.504386   
Timeline                                     -                llama3:7b zero-shot [15] (R) for    0.509636   
                                                              llama3:7b zero-shot [5] (R)  for    0.559546   
Texts                                        -                llama3:7b zero-shot [10] (R) for    0.508539   
                                                              llama3:7b zero-shot [15] (R) for    0.432749   
Stance                                       tf-idf           xgb                          for    0.654762   
                                             -                llama3:7b zero-shot          for    0.734426   
Timeline                                     bertabaporu-base xgb                          for    0.596273   
Bag of Followers                             tf-idf           LogisticRegression           for    0.538776   
Stacking BoM + BoFr                          -                LogisticRegression           for    0.562092   
Stacking Texts + BoM + BoF                   -                LogisticRegression           for    0.570833   
Stacking Texts + BoM + BoFr                  -                LogisticRegression           for    0.567850   
Stacking Texts + BoM                         -                LogisticRegression           for    0.567850   
Stacking Texts + BoF                         -                LogisticRegression           for    0.602362   
concat_Texts_Timeline                        tf-idf           xgb                          for    0.647541   
Timeline                                     -                bertabaporu-base             for    0.656766   
Stacking BoM + BoF                           -                LogisticRegression           for    0.626050   
Stacking BoF + BoFr                          -                LogisticRegression           for    0.557940   
Bag of Mentions                              tf-idf           LogisticRegression           for    0.582329   
Stacking Texts + BoM + BoF + BoFr            -                LogisticRegression           for    0.576763   
Stacking Texts + BoF + BoFr                  -                LogisticRegression           for    0.581673   
Stacking BoM + BoF + BoFr                    -                LogisticRegression           for    0.624473   
Bag of Friends                               tf-idf           LogisticRegression           for    0.626712   
Stacking Texts + Timeline + BoF + BoFr       -                LogisticRegression           for    0.649402   
Stacking Texts + Timeline + BoM + BoFr  

## Create table f1 macro

In [14]:
mask_f1 = [True if  "f1-score" in col else False for col in df_results_final.columns]
mask_macro = [True if  "macro avg" in col else False for col in df_results_final.index]

f1_macro_df = df_results_final.loc[mask_macro,mask_f1]
f1_macro_df[('f1-score','overall')] = f1_macro_df.mean(axis=1)

f1_macro_df

metric                                                                                                f1-score  \
target                                                                                                  Church   
text_col                                     vectorizer       estimator                    class                 
Stance                                       -                dummy                        macro avg  0.361407   
                                             tf-idf           xgb                          macro avg  0.702021   
                                             bertabaporu-base xgb                          macro avg  0.853801   
                                             -                bertabaporu-base             macro avg  0.866276   
                                                              llama3:7b zero-shot          macro avg  0.729458   
Texts                                        -                dummy                        macro avg  0.361407   
                                             tf-idf           xgb                          macro avg  0.596115   
                                             bertabaporu-base xgb                          macro avg  0.594754   
                                             -                bertabaporu-base             macro avg  0.586611   
                                                              bertabaporu-base (R)         macro avg  0.583417   
                                                              llama3:7b zero-shot [5] (R)  macro avg  0.499052   
                                                              llama3:7b zero-shot [10] (R) macro avg  0.561274   
                                                              llama3:7b zero-shot [15] (R) macro avg  0.503966   
Timeline                                     -                dummy                        macro avg  0.361407   
                                             tf-idf           xgb                          macro avg  0.699122   
                                             bertabaporu-base xgb                          macro avg  0.661773   
                                             -                bertabaporu-base             macro avg  0.652707   
                                                              bertabaporu-base (R)         macro avg  0.701990   
                                                              llama3:7b zero-shot [5] (R)  macro avg  0.605633   
                                                              llama3:7b zero-shot [10] (R) macro avg  0.563199   
                                                              llama3:7b zero-shot [15] (R) macro avg  0.598183   
Bag of Friends                               tf-idf           LogisticRegression           macro avg  0.635832   
Bag of Followers                             tf-idf           LogisticRegression           macro avg  0.609783   
Bag of Mentions                              tf-idf           LogisticRegression           macro avg  0.642593   
concat_Texts_Timeline                        tf-idf           xgb                          macro avg  0.702644   
Stacking Texts + Timeline                    -                LogisticRegression           macro avg  0.698596   
Stacking Texts + BoM                         -                LogisticRegression           macro avg  0.639975   
Stacking Texts + BoF                         -                LogisticRegression           macro avg  0.654804   
Stacking Texts + BoFr                        -                LogisticRegression           macro avg  0.599902   
Stacking Timeline + BoM                      -                LogisticRegression           macro avg  0.652754   
Stacking Timeline + BoF                      -                LogisticRegression           macro avg  0.684941   
Stacking Timeline + BoFr                     -                LogisticRegression           macro avg  0.699122   
Stacking B

### Table for docs

In [15]:
f1_report = f1_macro_df.copy()
f1_report

metric                                                                                                f1-score  \
target                                                                                                  Church   
text_col                                     vectorizer       estimator                    class                 
Stance                                       -                dummy                        macro avg  0.361407   
                                             tf-idf           xgb                          macro avg  0.702021   
                                             bertabaporu-base xgb                          macro avg  0.853801   
                                             -                bertabaporu-base             macro avg  0.866276   
                                                              llama3:7b zero-shot          macro avg  0.729458   
Texts                                        -                dummy                        macro avg  0.361407   
                                             tf-idf           xgb                          macro avg  0.596115   
                                             bertabaporu-base xgb                          macro avg  0.594754   
                                             -                bertabaporu-base             macro avg  0.586611   
                                                              bertabaporu-base (R)         macro avg  0.583417   
                                                              llama3:7b zero-shot [5] (R)  macro avg  0.499052   
                                                              llama3:7b zero-shot [10] (R) macro avg  0.561274   
                                                              llama3:7b zero-shot [15] (R) macro avg  0.503966   
Timeline                                     -                dummy                        macro avg  0.361407   
                                             tf-idf           xgb                          macro avg  0.699122   
                                             bertabaporu-base xgb                          macro avg  0.661773   
                                             -                bertabaporu-base             macro avg  0.652707   
                                                              bertabaporu-base (R)         macro avg  0.701990   
                                                              llama3:7b zero-shot [5] (R)  macro avg  0.605633   
                                                              llama3:7b zero-shot [10] (R) macro avg  0.563199   
                                                              llama3:7b zero-shot [15] (R) macro avg  0.598183   
Bag of Friends                               tf-idf           LogisticRegression           macro avg  0.635832   
Bag of Followers                             tf-idf           LogisticRegression           macro avg  0.609783   
Bag of Mentions                              tf-idf           LogisticRegression           macro avg  0.642593   
concat_Texts_Timeline                        tf-idf           xgb                          macro avg  0.702644   
Stacking Texts + Timeline                    -                LogisticRegression           macro avg  0.698596   
Stacking Texts + BoM                         -                LogisticRegression           macro avg  0.639975   
Stacking Texts + BoF                         -                LogisticRegression           macro avg  0.654804   
Stacking Texts + BoFr                        -                LogisticRegression           macro avg  0.599902   
Stacking Timeline + BoM                      -                LogisticRegression           macro avg  0.652754   
Stacking Timeline + BoF                      -                LogisticRegression           macro avg  0.684941   
Stacking Timeline + BoFr                     -                LogisticRegression           macro avg  0.699122   
Stacking B

In [16]:
f1_report.reset_index(drop=False, inplace=True)
f1_report

metric                                      text_col        vectorizer  \
target                                                                   
0                                             Stance                 -   
1                                             Stance            tf-idf   
2                                             Stance  bertabaporu-base   
3                                             Stance                 -   
4                                             Stance                 -   
5                                              Texts                 -   
6                                              Texts            tf-idf   
7                                              Texts  bertabaporu-base   
8                                              Texts                 -   
9                                              Texts                 -   
10                                             Texts                 -   
11                                             Texts                 -   
12                                             Texts                 -   
13                                          Timeline                 -   
14                                          Timeline            tf-idf   
15                                          Timeline  bertabaporu-base   
16                                          Timeline                 -   
17                                          Timeline                 -   
18                                          Timeline                 -   
19                                          Timeline                 -   
20                                          Timeline                 -   
21                                    Bag of Friends            tf-idf   
22                                  Bag of Followers            tf-idf   
23                                   Bag of Mentions            tf-idf   
24                             concat_Texts_Timeline            tf-idf   
25                         Stacking Texts + Timeline                 -   
26                              Stacking Texts + BoM                 -   
27                              Stacking Texts + BoF                 -   
28                             Stacking Texts + BoFr                 -   
29                           Stacking Timeline + BoM                 -   
30                           Stacking Timeline + BoF                 -   
31                          Stacking Timeline + BoFr                 -   
32                                Stacking BoM + BoF                 -   
33                               Stacking BoM + BoFr                 -   
34                               Stacking BoF + BoFr                 -   
35                   Stacking Texts + Timeline + BoM                 -   
36                   Stacking Texts + Timeline + BoF                 -   
37                  Stacking Texts + Timeline + BoFr                 -   
38                        Stacking Texts + BoM + BoF                 -   
39                       Stacking Texts + BoM + BoFr                 -   
40                       Stacking Texts + BoF + BoFr                 -   
41                     Stacking Timeline + BoM + BoF                 -   
42                    Stacking Timeline + BoM + BoFr                 -   
43                    Stacking Timeline + BoF + BoFr                 -   
44                         Stacking BoM + BoF + BoFr                 -   
45             Stacking Texts + Timeline + BoM + BoF                 -   
46            Stacking Texts + Timeline + BoM + BoFr                 -   
47            Stacking Texts + Timeline + BoF + BoFr                 -   
48                 Stacking Texts + BoM + BoF + BoFr                 -   
49              Stacking Timeline + BoM + BoF + BoFr                 -   
50      Stacking Texts + Timeline + BoM + BoF + BoFr                 -   

metric                     estimator      class  f1-score                    

In [17]:
f1_report.columns

MultiIndex([(  'text_col',          ''),
            ('vectorizer',          ''),
            ( 'estimator',          ''),
            (     'class',          ''),
            (  'f1-score',    'Church'),
            (  'f1-score', 'Bolsonaro'),
            (  'f1-score',   'Hydrox.'),
            (  'f1-score',   'Sinovac'),
            (  'f1-score',  'Globo TV'),
            (  'f1-score',      'Lula'),
            (  'f1-score',   'overall')],
           names=['metric', 'target'])

In [18]:
new_columns = [col[0] if col[1] == '' else col[1] for col in f1_report.columns]
new_columns

['text_col',
 'vectorizer',
 'estimator',
 'class',
 'Church',
 'Bolsonaro',
 'Hydrox.',
 'Sinovac',
 'Globo TV',
 'Lula',
 'overall']

In [19]:
f1_report.columns = new_columns
f1_report

,text_col,vectorizer,estimator,class,Church,Bolsonaro,Hydrox.,Sinovac,Globo TV,Lula,overall
0,Stance,-,dummy,macro avg,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
1,Stance,tf-idf,xgb,macro avg,0.702021,0.595312,0.734366,0.752640,0.665243,0.654412,0.683999
2,Stance,bertabaporu-base,xgb,macro avg,0.853801,0.625289,0.830948,0.808036,0.781858,0.766993,0.777821
3,Stance,-,bertabaporu-base,macro avg,0.866276,0.737297,0.848144,0.834984,0.860252,0.789802,0.822792
4,Stance,-,llama3:7b zero-shot,macro avg,0.729458,0.462857,0.638889,0.578779,0.770678,0.699381,0.646674
5,Texts,-,dummy,macro avg,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
6,Texts,tf-idf,xgb,macro avg,0.596115,0.501220,0.603727,0.670805,0.545620,0.579427,0.582819
7,Texts,bertabaporu-base,xgb,macro avg,0.594754,0.495578,0.609524,0.660542,0.534455,0.569597,0.577408
8,Texts,-,bertabaporu-base,macro avg,0.586611,0.462857,0.446504,0.608972,0.507381,0.482520,0.515808
9,Texts,-,bertabaporu-base (R),macro avg,0.583417,0.462857,0.570531,0.637369,0.586967,0.602167,0.573885


In [20]:
f1_report.drop(['class'],axis = 1, inplace=True)
f1_report

,text_col,vectorizer,estimator,Church,Bolsonaro,Hydrox.,Sinovac,Globo TV,Lula,overall
0,Stance,-,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
1,Stance,tf-idf,xgb,0.702021,0.595312,0.734366,0.752640,0.665243,0.654412,0.683999
2,Stance,bertabaporu-base,xgb,0.853801,0.625289,0.830948,0.808036,0.781858,0.766993,0.777821
3,Stance,-,bertabaporu-base,0.866276,0.737297,0.848144,0.834984,0.860252,0.789802,0.822792
4,Stance,-,llama3:7b zero-shot,0.729458,0.462857,0.638889,0.578779,0.770678,0.699381,0.646674
5,Texts,-,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
6,Texts,tf-idf,xgb,0.596115,0.501220,0.603727,0.670805,0.545620,0.579427,0.582819
7,Texts,bertabaporu-base,xgb,0.594754,0.495578,0.609524,0.660542,0.534455,0.569597,0.577408
8,Texts,-,bertabaporu-base,0.586611,0.462857,0.446504,0.608972,0.507381,0.482520,0.515808
9,Texts,-,bertabaporu-base (R),0.583417,0.462857,0.570531,0.637369,0.586967,0.602167,0.573885


In [21]:
f1_report.insert(
    1, 
    "classifier", 
    f1_report.apply(
        lambda x: f"{x['vectorizer']} + {x['estimator']}" if x['vectorizer'] != '-' else x['estimator'],
        axis = 1
        ).to_list()

)
f1_report.drop(['estimator', 'vectorizer'],axis =1, inplace = True)
f1_report

,text_col,classifier,Church,Bolsonaro,Hydrox.,Sinovac,Globo TV,Lula,overall
0,Stance,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
1,Stance,tf-idf + xgb,0.702021,0.595312,0.734366,0.752640,0.665243,0.654412,0.683999
2,Stance,bertabaporu-base + xgb,0.853801,0.625289,0.830948,0.808036,0.781858,0.766993,0.777821
3,Stance,bertabaporu-base,0.866276,0.737297,0.848144,0.834984,0.860252,0.789802,0.822792
4,Stance,llama3:7b zero-shot,0.729458,0.462857,0.638889,0.578779,0.770678,0.699381,0.646674
5,Texts,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
6,Texts,tf-idf + xgb,0.596115,0.501220,0.603727,0.670805,0.545620,0.579427,0.582819
7,Texts,bertabaporu-base + xgb,0.594754,0.495578,0.609524,0.660542,0.534455,0.569597,0.577408
8,Texts,bertabaporu-base,0.586611,0.462857,0.446504,0.608972,0.507381,0.482520,0.515808
9,Texts,bertabaporu-base (R),0.583417,0.462857,0.570531,0.637369,0.586967,0.602167,0.573885


In [22]:
f1_report.rename({"text_col":"input"}, axis = 1, inplace=True)
f1_report

,input,classifier,Church,Bolsonaro,Hydrox.,Sinovac,Globo TV,Lula,overall
0,Stance,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
1,Stance,tf-idf + xgb,0.702021,0.595312,0.734366,0.752640,0.665243,0.654412,0.683999
2,Stance,bertabaporu-base + xgb,0.853801,0.625289,0.830948,0.808036,0.781858,0.766993,0.777821
3,Stance,bertabaporu-base,0.866276,0.737297,0.848144,0.834984,0.860252,0.789802,0.822792
4,Stance,llama3:7b zero-shot,0.729458,0.462857,0.638889,0.578779,0.770678,0.699381,0.646674
5,Texts,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
6,Texts,tf-idf + xgb,0.596115,0.501220,0.603727,0.670805,0.545620,0.579427,0.582819
7,Texts,bertabaporu-base + xgb,0.594754,0.495578,0.609524,0.660542,0.534455,0.569597,0.577408
8,Texts,bertabaporu-base,0.586611,0.462857,0.446504,0.608972,0.507381,0.482520,0.515808
9,Texts,bertabaporu-base (R),0.583417,0.462857,0.570531,0.637369,0.586967,0.602167,0.573885


In [23]:
#f1_report.set_index(['input'],inplace=True)
#f1_report.drop('input', axis = 1,inplace=True)

In [24]:
f1_report.input.value_counts()

input
Texts                                           8
Timeline                                        8
Stance                                          5
Stacking Timeline + BoM + BoF + BoFr            1
Stacking Texts + BoM + BoF + BoFr               1
Stacking Texts + Timeline + BoF + BoFr          1
Stacking Texts + Timeline + BoM + BoFr          1
Stacking Texts + Timeline + BoM + BoF           1
Stacking Texts + Timeline + BoF                 1
Stacking Timeline + BoF + BoFr                  1
Stacking Timeline + BoM + BoFr                  1
Stacking Timeline + BoM + BoF                   1
Stacking Texts + BoF + BoFr                     1
Stacking Texts + BoM + BoFr                     1
Stacking Texts + BoM + BoF                      1
Stacking Texts + Timeline + BoFr                1
Stacking BoM + BoF + BoFr                       1
Stacking BoF + BoFr                             1
Stacking Texts + Timeline + BoM                 1
Stacking BoM + BoFr                         

In [25]:
f1_report

,input,classifier,Church,Bolsonaro,Hydrox.,Sinovac,Globo TV,Lula,overall
0,Stance,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
1,Stance,tf-idf + xgb,0.702021,0.595312,0.734366,0.752640,0.665243,0.654412,0.683999
2,Stance,bertabaporu-base + xgb,0.853801,0.625289,0.830948,0.808036,0.781858,0.766993,0.777821
3,Stance,bertabaporu-base,0.866276,0.737297,0.848144,0.834984,0.860252,0.789802,0.822792
4,Stance,llama3:7b zero-shot,0.729458,0.462857,0.638889,0.578779,0.770678,0.699381,0.646674
5,Texts,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
6,Texts,tf-idf + xgb,0.596115,0.501220,0.603727,0.670805,0.545620,0.579427,0.582819
7,Texts,bertabaporu-base + xgb,0.594754,0.495578,0.609524,0.660542,0.534455,0.569597,0.577408
8,Texts,bertabaporu-base,0.586611,0.462857,0.446504,0.608972,0.507381,0.482520,0.515808
9,Texts,bertabaporu-base (R),0.583417,0.462857,0.570531,0.637369,0.586967,0.602167,0.573885


In [26]:
str_latex = generate_latex_with_multirow_and_bold(f1_report)

In [27]:
print(str_latex)

\begin{table}[H]\scriptsize\centering\begin{tabular}{ll|rrrrrrr}
\toprule
input & classifier & Church & Bolsonaro & Hydrox. & Sinovac & Globo TV & Lula & overall \\ 
\midrule
\multirow{2}{*}{Stance} & dummy & 0.36 & 0.46 & 0.33 & 0.35 & 0.37 & 0.34 & 0.37 \\ 
& tf-idf + xgb & 0.70 & 0.60 & 0.73 & 0.75 & 0.67 & 0.65 & 0.68 \\ 
& bertabaporu-base + xgb & 0.85 & 0.63 & 0.83 & 0.81 & 0.78 & 0.77 & 0.78 \\ 
& \textbf{bertabaporu-base} & \textbf{0.87} & \textbf{0.74} & \textbf{0.85} & \textbf{0.83} & \textbf{0.86} & \textbf{0.79} & \textbf{0.82} \\ 
& llama3:7b zero-shot & 0.73 & 0.46 & 0.64 & 0.58 & 0.77 & 0.70 & 0.65 \\ 
\cmidrule(lr){1-9}
\multirow{2}{*}{Texts} & dummy & 0.36 & 0.46 & 0.33 & 0.35 & 0.37 & 0.34 & 0.37 \\ 
& \textbf{tf-idf + xgb} & \textbf{0.60} & \textbf{0.50} & \textbf{0.60} & \textbf{0.67} & \textbf{0.55} & \textbf{0.58} & \textbf{0.58} \\ 
& bertabaporu-base + xgb & 0.59 & 0.50 & 0.61 & 0.66 & 0.53 & 0.57 & 0.58 \\ 
& bertabaporu-base & 0.59 & 0.46 & 0.45 & 0.61 & 0.51 